In [3]:
import findspark
findspark.init()
import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df = spark.sql('''select 'spark' as hello ''')
df.show()

23/11/23 13:34:13 WARN Utils: Your hostname, codespaces-3ca734 resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
23/11/23 13:34:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/23 13:34:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+-----+
|hello|
+-----+
|spark|
+-----+



23/11/23 13:34:23 WARN JavaUtils: Attempt to delete using native Unix OS command failed for path = /tmp/spark-056da2cf-de87-47d0-b661-5599633ea461/userFiles-a9253072-75b5-4eda-933c-3f2138e3feee. Falling back to Java IO way
java.io.IOException: Failed to delete: /tmp/spark-056da2cf-de87-47d0-b661-5599633ea461/userFiles-a9253072-75b5-4eda-933c-3f2138e3feee
	at org.apache.spark.network.util.JavaUtils.deleteRecursivelyUsingUnixNative(JavaUtils.java:173)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:109)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:90)
	at org.apache.spark.util.SparkFileUtils.deleteRecursively(SparkFileUtils.scala:121)
	at org.apache.spark.util.SparkFileUtils.deleteRecursively$(SparkFileUtils.scala:120)
	at org.apache.spark.util.Utils$.deleteRecursively(Utils.scala:1126)
	at org.apache.spark.SparkEnv.stop(SparkEnv.scala:108)
	at org.apache.spark.SparkContext.$anonfun$stop$25(SparkContext.scala:2310)
	at org.apac